In [2]:
import json
import pandas as pd
import altair as alt
import geopandas as gpd
import re
from data_cleaner import *
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [3]:
raw = pd.read_csv("datatest.csv")

In [4]:
regions = raw["Region"].value_counts()
countries = raw["Country"].value_counts()
event_types = raw["event_type"].value_counts()
sub_event_types = raw["sub_event_type"].value_counts()

In [5]:
violence_object = {}

for region, value in regions.items():
    country_subset = raw[raw["Region"] == region]
    violence_object[region] = {}
    violence_object[region]["violent_events"] = country_subset.count()
    
    for row in country_subset.items():
        country = row["Country"]
        just_country = raw[raw["Country"] == country] 
        violence_object[region][country] = {}
        violence_object[region][country]["violent_events"] = just_country.count()
    
for region in violence_object:
    violence_subset = '1'
    


TypeError: tuple indices must be integers or slices, not str

In [6]:
violence_object = {}

for region, value in regions.items():
    country_subset = raw[raw["Region"] == region]
    violence_object[region] = {}
    violence_object[region]["name"] = region
    violence_object[region]["violent_events"] = value
    region_countries = country_subset["Country"].value_counts()

    for country, value in region_countries.items():
        violence_object[region][country] = {}
        violence_object[region][country]["name"] = country
        violence_object[region][country]["violent_events"] = value

In [ ]:
violence_object

In [ ]:
with open("samplebubble.json", "w") as outfile: 
    json.dump(violence_object, outfile)

In [8]:
#different data shape:
raw2 = pd.read_csv("datatest.csv")

In [10]:
testing = raw2.value_counts()

In [17]:
testing.iloc[0]

np.int64(5)

In [ ]:
violence_array = []

for id, value in testing.items():
    row = {}
    row["Country"] = id[0]
    row['Region'] = id[1]
    row['event_type'] = id[2]
    row['sub_event_type'] = id[3]
    row['violent_events'] = value
    violence_array.append(row)

violence_array

In [21]:
with open("arraybubble.json", "w") as outfile: 
    json.dump(violence_array, outfile)

In [28]:
real = pd.read_csv("../data/acled_2023.csv")

In [ ]:
acled = real[real["event_type"] != "Protests"]
acled = acled[acled["sub_event_type"] != "Headquarters or base established"]
acled = acled[acled["sub_event_type"] != "Other"]
acled = acled[acled["sub_event_type"] != "Change to group/activity"]
acled = acled[acled["sub_event_type"] != "Agreement"]
acled = acled[acled["sub_event_type"] != "Non-violent transfer of territory"]
acled = acled[acled['year'] == 2023]
good_cols = acled[['region','country','event_type']]

counter = good_cols.value_counts()

acled_array = []

for id, value in counter.items():
    row = {}
    row["Country"] = id[1]
    row['Region'] = id[0]
    row['event_type'] = id[2]
    row['violent_events'] = value
    acled_array.append(row)

acled_array



In [34]:
with open("acledArray.json", "w") as outfile: 
    json.dump(acled_array, outfile)